In [1]:
import os
import xarray as xr
import xdggs
import zarr
os.environ['DGGRID_PATH']='/home/dick/micromamba/envs/test/bin/dggrid'
from xdggs_dggrid4py.IGEO7 import IGEO7Index

## A simple step by step howto
This notebook showcases converting a 2D array dataset to a 1D dataset with DGGS (IGEO7) as an index. 
- prepare data
- conversion from 2D coordinates to DGGS cell id
- some use cases on DGGS

### Prepare Data

In [2]:
# data source : https://stacindex.org/catalogs/cubes-and-clouds-snow-cover#/BXM1FNA8yFQD4Mwf6jW28EX3DqXfYKUah?t=2
data = xr.open_dataset("https://esa.pages.eox.at/cubes-and-clouds-catalog/contributions/valentin.printemps@gmail.com/openEO_uint8.tif", band_as_variable=True, engine='rasterio',chunks='auto')
data

<xarray.Dataset> Size: 126kB
Dimensions:      (x: 150, y: 206)
Coordinates:
  * x            (x) float64 1kB 3.151e+05 3.151e+05 ... 3.165e+05 3.166e+05
  * y            (y) float64 2kB 4.939e+06 4.939e+06 ... 4.937e+06 4.937e+06
    spatial_ref  int64 8B ...
Data variables:
    band_1       (y, x) float32 124kB dask.array<chunksize=(206, 150), meta=np.ndarray>
Attributes:
    institution:                 openEO platform - Geotrellis backend: 0.59.0a1
    PROCESSING_SOFTWARE:         0.59.0a1
    {TIFFTAG_IMAGEDESCRIPTION}:  SnowCoveredArea_0=nosnow_1=snow_2-nodatavalu...
    AREA_OR_POINT:               Area

We need to specify the attributes of DGGS for conversion; the content varies from different DGGS.
The full attributes content of IGEO7 is :
```python
    {    "grid_name": "igeo7",  
             "level": -1,  
          "src_epsg": "EPSG:32632", 
           "method" : "nearestpoint", # centroid or nearestpoint 
       "coordinate" : ["x","y"], # what are the coordinate name in the xarray
               "mp" : 1, # for multiprocessing, right now only for linux with pymp-pypi install manually
             "chunk" : (100,100),
    }
```
After that, we assign the attributes to one of the current coordinates, for this example, either x or y.

In [3]:
attrs={"grid_name": "igeo7",
        "level": -1,
        "src_epsg": "EPSG:32632",
        "method" : "nearestpoint",
        "coordinate" : ["x","y"]}
data['y'].attrs=attrs

### Conversion from 2D coordinates to DGGS cell id
To peform the conversion, we use the stack function from xarray to create a new Index with the class `IGEO7Index` provided.

In [4]:
%%time
dggs_data = data.stack(cell_ids=("x", "y"), index_cls=IGEO7Index)

x shape: ((150,)), y shape: ((206,))
Calculate Auto resolution
4937085.0,315065.0,4939135.0,316555.0
Total Bounds (EPSG:32632): [ 315065. 4937085.  316555. 4939135.]
Total Bounds (wgs84): [ 6.67045856 44.56335938  6.68994264 44.58218062]
Total Bounds Area (km^2): 3.2299606105177596
Area per center point (km^2): 0.000104529469596044
Auto resolution : 14
--- Multiprocessing 1 ---


  0%|          | 0/1 [00:00<?, ?it/s]

---Generate Cell ID with resolution 14 by nearestpoint method, x batch:1, y batch:1---
cell generation time: (0.9296696186065674)
Cell ID calcultion completed, unique cell id :28463
CPU times: user 241 ms, sys: 8.31 ms, total: 249 ms
Wall time: 1.03 s


/home/dick/micromamba/envs/test/lib/python3.13/site-packages/pyogrio/raw.py:198: RuntimeWarning: driver FlatGeobuf does not support open option DRIVER
  return ogr_read(


After conversion, the dataset is transformed into one dimension, as shown below.
However, there are some drawbacks to using the stack : 
- After conversion, the index can be used immediately, as it becomes an multi-index `(cell_ids, x, y)`
- The attributes can't be carried to the newly created index

So we have to : 
1. assign back the attributes manually to cell_ids
2. save the dataset to disk and load it back to decomposit the multi-index

In [5]:
dggs_data

<xarray.Dataset> Size: 865kB
Dimensions:      (cell_ids: 30900)
Coordinates:
  * x            (cell_ids) float64 247kB 3.151e+05 3.151e+05 ... 3.166e+05
  * y            (cell_ids) float64 247kB 4.939e+06 4.939e+06 ... 4.937e+06
    spatial_ref  int64 8B ...
  * cell_ids     (cell_ids) object 247kB '0061506550626361' ... '006150655260...
Data variables:
    band_1       (cell_ids) float32 124kB dask.array<chunksize=(30900,), meta=np.ndarray>
Indexes:
  ┌ cell_ids  ISEAIndex(grid_name=IGEO7, level=14)
  │ x
  └ y
Attributes:
    institution:                 openEO platform - Geotrellis backend: 0.59.0a1
    PROCESSING_SOFTWARE:         0.59.0a1
    {TIFFTAG_IMAGEDESCRIPTION}:  SnowCoveredArea_0=nosnow_1=snow_2-nodatavalu...
    AREA_OR_POINT:               Area

In [7]:
# Very important ! copy the attributes to cell_ids
dggs_data.cell_ids.attrs = dggs_data.xindexes.get('cell_ids')._grid.to_dict()
# save it to zarr
compressor = zarr.Blosc(cname="zstd", clevel=3, shuffle=2)
dggs_data.to_zarr(f'dggs_data.zar',encoding={"band_1": {"compressor": compressor},"cell_ids": {"compressor": compressor}})

After we load the dataset from the disk again, the index becomes a single index, but it is loaded as PandasIndex.
We can use the `xdggs.decode` function to re-initialize it to the IGEO7 index.

In [8]:
dggs_zarr = xr.open_zarr('./dggs_data.zar')
dggs_zarr 

<xarray.Dataset> Size: 865kB
Dimensions:      (cell_ids: 30900)
Coordinates:
  * cell_ids     (cell_ids) object 247kB '0061506550626361' ... '006150655260...
    x            (cell_ids) float64 247kB dask.array<chunksize=(30900,), meta=np.ndarray>
    y            (cell_ids) float64 247kB dask.array<chunksize=(30900,), meta=np.ndarray>
Data variables:
    band_1       (cell_ids) float32 124kB dask.array<chunksize=(30900,), meta=np.ndarray>
    spatial_ref  int64 8B ...
Attributes:
    AREA_OR_POINT:               Area
    PROCESSING_SOFTWARE:         0.59.0a1
    institution:                 openEO platform - Geotrellis backend: 0.59.0a1
    {TIFFTAG_IMAGEDESCRIPTION}:  SnowCoveredArea_0=nosnow_1=snow_2-nodatavalu...

In [9]:
dggs_zarr = xdggs.decode(dggs_zarr) 
dggs_zarr

<xarray.Dataset> Size: 865kB
Dimensions:      (cell_ids: 30900)
Coordinates:
  * cell_ids     (cell_ids) object 247kB '0061506550626361' ... '006150655260...
    x            (cell_ids) float64 247kB dask.array<chunksize=(30900,), meta=np.ndarray>
    y            (cell_ids) float64 247kB dask.array<chunksize=(30900,), meta=np.ndarray>
Data variables:
    band_1       (cell_ids) float32 124kB dask.array<chunksize=(30900,), meta=np.ndarray>
    spatial_ref  int64 8B ...
Indexes:
    cell_ids  ISEAIndex(grid_name=IGEO7, level=14)
Attributes:
    AREA_OR_POINT:               Area
    PROCESSING_SOFTWARE:         0.59.0a1
    institution:                 openEO platform - Geotrellis backend: 0.59.0a1
    {TIFFTAG_IMAGEDESCRIPTION}:  SnowCoveredArea_0=nosnow_1=snow_2-nodatavalu...

In [10]:
dggs_zarr = dggs_zarr.drop_vars(['x','y'])
dggs_zarr.dggs.sel_latlon(latitude=[44.56375059,44.56369803],longitude=[6.68935115])

<xarray.Dataset> Size: 32B
Dimensions:      (cell_ids: 2)
Coordinates:
  * cell_ids     (cell_ids) object 16B '0061506552601556' '0061506552601550'
Data variables:
    band_1       (cell_ids) float32 8B dask.array<chunksize=(2,), meta=np.ndarray>
    spatial_ref  int64 8B ...
Indexes:
    cell_ids  ISEAIndex(grid_name=IGEO7, level=14)
Attributes:
    AREA_OR_POINT:               Area
    PROCESSING_SOFTWARE:         0.59.0a1
    institution:                 openEO platform - Geotrellis backend: 0.59.0a1
    {TIFFTAG_IMAGEDESCRIPTION}:  SnowCoveredArea_0=nosnow_1=snow_2-nodatavalu...

In [11]:
dggs_zarr.dggs.cell_centers()

/home/dick/micromamba/envs/test/lib/python3.13/site-packages/pyogrio/raw.py:198: RuntimeWarning: driver FlatGeobuf does not support open option DRIVER
  return ogr_read(


<xarray.Dataset> Size: 455kB
Dimensions:    (cell_ids: 28463)
Coordinates:
    latitude   (cell_ids) float64 228kB 44.56 44.56 44.56 ... 44.56 44.56 44.56
    longitude  (cell_ids) float64 228kB 6.69 6.689 6.689 ... 6.671 6.671 6.671
Dimensions without coordinates: cell_ids
Data variables:
    *empty*

In [12]:
dggs_zarr.dggs.cell_boundaries()

/home/dick/micromamba/envs/test/lib/python3.13/site-packages/pyogrio/raw.py:198: RuntimeWarning: driver FlatGeobuf does not support open option DRIVER
  return ogr_read(


<xarray.DataArray (cell_ids: 30900)> Size: 247kB
<class 'xarray.core.extension_array.PandasExtensionArray'>(array=<GeometryArray>
[<POLYGON ((6.671 44.582, 6.671 44.582, 6.67 44.582, 6.67 44.582, 6.671 44.58...>,
 <POLYGON ((6.671 44.582, 6.671 44.582, 6.671 44.582, 6.671 44.582, 6.671 44....>,
 <POLYGON ((6.671 44.582, 6.671 44.582, 6.671 44.582, 6.671 44.582, 6.671 44....>,
 <POLYGON ((6.671 44.582, 6.671 44.582, 6.671 44.582, 6.671 44.582, 6.671 44....>,
 <POLYGON ((6.671 44.582, 6.671 44.582, 6.671 44.582, 6.671 44.582, 6.671 44....>,
 <POLYGON ((6.671 44.582, 6.671 44.582, 6.671 44.582, 6.671 44.582, 6.671 44....>,
 <POLYGON ((6.671 44.582, 6.671 44.582, 6.671 44.582, 6.671 44.582, 6.671 44....>,
 <POLYGON ((6.671 44.582, 6.671 44.582, 6.671 44.582, 6.671 44.582, 6.671 44....>,
 <POLYGON ((6.672 44.582, 6.671 44.582, 6.671 44.582, 6.671 44.582, 6.671 44....>,
 <POLYGON ((6.672 44.582, 6.672 44.582, 6.672 44.582, 6.672 44.582, 6.672 44....>,
 ...
 <POLYGON ((6.689 44.564, 6.689 44.564, 6.689 44.564, 6.689 44.564, 6.689 44....>,
 <POLYGON ((6.689 44.564, 6.689 44.564, 6.689 44.564, 6.689 44.564, 6.689 44....>,
 <POLYGON ((6.689 44.564, 6.689 44.564, 6.689 44.564, 6.689 44.564, 6.689 44....>,
 <POLYGON ((6.689 44.564, 6.689 44.564, 6.689 44.564, 6.689 44.564, 6.689 44....>,
 <POLYGON ((6.689 44.564, 6.689 44.564, 6.689 44.564, 6.689 44.564, 6.689 44....>,
 <POLYGON ((6.689 44.564, 6.689 44.564, 6.689 44.564, 6.689 44.564, 6.689 44....>,
 <POLYGON ((6.69 44.564, 6.69 44.564, 6.689 44.564, 6.689 44.564, 6.69 44.564...>,
 <POLYGON ((6.69 44.564, 6.69 44.564, 6.69 44.564, 6.69 44.564, 6.69 44.564, ...>,
 <POLYGON ((6.69 44.564, 6.69 44.564, 6.69 44.564, 6.69 44.564, 6.69 44.564, ...>,
 <POLYGON ((6.69 44.564, 6.69 44.564, 6.69 44.564, 6.69 44.564, 6.69 44.564, ...>]
Length: 30900, dtype: geometry)
Coordinates:
  * cell_ids  (cell_ids) object 247kB '0061506550626361' ... '0061506552601535'